# Tweet Sentiment Classification with Fine-Tuned GPT-2
*A transformer-based solution for decoding mood on social media*

## Import Libraries

Libraries required for data processing and modeling are imported.

This project runs in Google Colab to make use of the free GPU support.
Start by mounting Google Drive to access the project files. If this is your first time running the project, clone the repository. Otherwise, pull the latest changes to stay up to date.
Once that’s done, run the setup script to install any dependencies and make sure the environment is ready to go.

In [ ]:
# Mount Drive on colab manually
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# If the repo doesn't exist in Google Drive, clone it
# !git clone https://github.com/herrerovir/gpt2-tweet-sentiment

# If the repo does exist:
%cd /content/drive/MyDrive/Colab_Projects/gpt2-tweet-sentiment
!git pull

In [ ]:
# Import config file
import config
config.setup(subdir = "notebooks")
from config import *